In [1]:
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
import urllib

In [4]:
#The 'query' key contains the search terms exactly
args = {'query':'honey bee health','apiKey':'582c2ee616e8c4119ae4201ce59a5e06'}
r = requests.get('http://api.elsevier.com/content/search/scopus?'+urllib.urlencode(args))
data = r.json()

In [13]:
login = open("./LoginInfo.txt","r")
user = login.readline()
password = login.readline()
login.close()

In [14]:
abstractPage = requests.get("https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85039924208&origin=inward",auth = HTTPBasicAuth(user,password))
bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
abstractText = bSoupObj.find("section",{"id":"authorKeywords"}).findAll("span",{"class":"badges"})
# print abstractText
print abstractText[1].text

Kalman filter


In [15]:
sKeywords = ""
for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
    sKeywords+= word.text+", "
print sKeywords[:-2]

Color, Food products, Food storage, Hough transforms, Elliptical shapes, Image moments, Object discrimination, Pollen counting, Pollen discriminations, Receiver operating characteristics, Tracking models, Video monitoring, Kalman filters


In [16]:
f = open("../data/paperTable.tsv","w")
entry_size = len(data['search-results']['entry'])
start_point = 25
while(entry_size>0):
    for entry in data['search-results']['entry']:
        #gets abstract text
        abstractPage = requests.get(entry['link'][2]['@href'],auth = HTTPBasicAuth(user,password))
        bSoupObj = BeautifulSoup(abstractPage.content,"html5lib")
        if(len(bSoupObj.findAll("p"))>1):
            abstractText = bSoupObj.findAll("p")[1].text
        
        #creates lists of keywords, seperated into author keywords and scopus assigned keywords
            aKeywords = ""
            sKeywords = ""
            if bSoupObj.find("section",{"id":"authorKeywords"}) is not None:
                for word in bSoupObj.find("section",{"id":"authorKeywords"}).findAll("li"):
                    aKeywords+= word.text+", "
            if bSoupObj.find("section",{"id":"indexedKeywords"}) is not None:
                for word in bSoupObj.find("section",{"id":"indexedKeywords"}).findAll("span"):
                    sKeywords+= word.text+", "

            #prints title, abstract, tags, and date created
            if abstractText.encode("utf8")=="[No abstract available]":
                print("no abs")
            else:
                f.write(entry['dc:title'].encode("utf8")+"\t"+abstractText.encode("utf8")+"\t"+aKeywords[:-2].encode("utf8")+"\t"+sKeywords[:-2].encode("utf8")+"\t"+entry['prism:coverDate'].encode("utf8")+"\n")
    data = requests.get('http://api.elsevier.com/content/search/scopus?query=Honey-Bee-Health&apiKey=eb73f64c03599119a51db065202c555f&start='+str(start_point)).json()
    start_point+=25
    entry_size+=len(data['search-results']['entry'])
f.close()

no abs
no abs
no abs
no abs
no abs


KeyboardInterrupt: 

In [12]:
len(data['search-results']['entry'])


25